In [8]:
import io
from pathlib import Path
import pickle
from PIL import Image

import numpy as np

import requests
from tqdm.notebook import tqdm

In [9]:
with open('name_to_avatar_map.pickle', 'rb') as f:
    name_to_avatar_map = pickle.load(f)
len(name_to_avatar_map)

632

In [10]:
avdir = Path('avatars/')
default = avdir / 'default'
nondefault = avdir / 'nondefault'
for d in [avdir, default, nondefault]:
    if not d.is_dir():
        d.mkdir()

Test run - first one is a custom, second is a default

In [11]:
nm, avurl = next(iter(name_to_avatar_map.items()))

r1 = requests.get(avurl)
im1 = Image.open(io.BytesIO(r1.content))

arr1 = np.array(im1)
flattenedarr1 = arr1.reshape(arr1.size//arr1.shape[-1], arr1.shape[-1])
unique_colors1 = np.unique(flattenedarr1, axis=0)
unique_colors1.shape[0], r1.headers['Content-Type']

(20560, 'image/jpeg')

In [12]:
nm = 'larrybradley'
avurl = name_to_avatar_map[nm]

r1 = requests.get(avurl)
im1 = Image.open(io.BytesIO(r1.content))

arr1 = np.array(im1)
flattenedarr1 = arr1.reshape(arr1.size//arr1.shape[-1], arr1.shape[-1])
unique_colors1 = np.unique(flattenedarr1, axis=0)
unique_colors1.shape[0], r1.headers['Content-Type']

(2, 'image/png')

In [13]:
avurl.split('/')[-1].split('?')[0]

'4992897'

In [14]:
for nm, avurl in tqdm(list(name_to_avatar_map.items())):
    r = requests.get(avurl)
    im = Image.open(io.BytesIO(r.content))
    arr = np.array(im)
    flattenedarr = arr.reshape(arr.size//arr.shape[-1], arr.shape[-1])
    unique_colors = np.unique(flattenedarr, axis=0)

    fnm = nm + '.' + r.headers['Content-Type'].split('/')[-1]

    if unique_colors.shape[0] == 2:
        outfn = default / fnm
    else:
        outfn = nondefault / fnm

    with outfn.open('wb') as f:
        f.write(r.content)
    

  0%|          | 0/632 [00:00<?, ?it/s]